Movie Recommendation Engine

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
pwd

'C:\\Users\\Admin\\Desktop\\Beautiful'

In [4]:
#Loading and Storing the data 
df = pd.read_csv(r"C:\Users\Admin\Desktop\Beautiful\Recommendation_System\movie_data.csv")
df.head(3)

,imdb_title_id,title,original_title,year,reviews_from_critics,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,reviews_from_users
0,tt0000009,Miss Jerry,Miss Jerry,1894,2.0,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,1.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,7.0,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,2.0,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,5.0


In [6]:
df.shape

(69999, 17)

In [7]:
df.isnull().sum()

imdb_title_id              0
title                      0
original_title             0
year                       0
reviews_from_critics    9589
genre                      0
duration                   0
country                   23
language                 524
director                  44
writer                  1017
production_company      3376
actors                    40
description             1891
avg_vote                   0
votes                      0
reviews_from_users      5388
dtype: int64

In [8]:
missing_value=["N/a","na","np.nan"]
df = pd.read_csv(r"C:\Users\Admin\Desktop\Beautiful\Recommendation_System\movie_data.csv", na_values= missing_value)

In [9]:
df.isnull().sum()

imdb_title_id              0
title                      0
original_title             0
year                       0
reviews_from_critics    9589
genre                      0
duration                   0
country                   23
language                 524
director                  44
writer                  1017
production_company      3376
actors                    40
description             1891
avg_vote                   0
votes                      0
reviews_from_users      5388
dtype: int64

In [12]:
df.dropna(how = 'any').shape

(54526, 17)

In [13]:
df.dropna(how = 'all').shape

(69999, 17)

In [21]:
new_df = df.dropna()
new_df

,imdb_title_id,title,original_title,year,reviews_from_critics,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,reviews_from_users
0,tt0000009,Miss Jerry,Miss Jerry,1894,2.0,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,1.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,7.0,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,7.0
3,tt0002101,Cleopatra,Cleopatra,1912,3.0,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,25.0
4,tt0002130,L'Inferno,L'Inferno,1911,14.0,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,31.0
5,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,5.0,"Biography, Drama",60,USA,English,Sidney Olcott,Gene Gauntier,Kalem Company,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,484,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69994,tt3136112,Shelter,Shelter,2014,29.0,Drama,105,USA,"English, French",Paul Bettany,Paul Bettany,Bifrost Pictures,"Adolfo Mendez-Nouel, Anthony Mackie, Kevin Hof...",Hannah and Tahir fall in love while homeless o...,6.5,4322,32.0
69995,tt3136646,Dukhless 2,Dukhless 2,2015,2.0,Drama,103,Russia,Russian,Roman Prygunov,"Fuad Ibragimbekov, Mikhail Idov",Art Pictures Studio,"Danila Kozlovsky, Mariya Andreeva, Milos Bikov...",Max Andreev thought the best place to start a ...,6.2,1555,2.0
69996,tt3136752,Le dernier diamant,Le dernier diamant,2014,22.0,"Crime, Drama, Thriller",108,"France, Luxembourg, Belgium",French,Eric Barbier,"Eric Barbier, Marie Eynard",Vertigo,"Yvan Attal, Bérénice Bejo, Jean-François Stéve...",Simon has just been released from prison and i...,6.3,1497,9.0
69997,tt3137546,Legacy,Legacy,2015,1.0,"Comedy, Drama",95,UK,English,"Davie Fairbanks, Marc Small","Davie Fairbanks, Marc Small",Unstoppable Entertainment,"Franz Drameh, Amy Tyger, McKell David, Jacob C...",What do you do when the party of the year kick...,4.5,331,4.0


In [22]:
new_df.isnull().values.any()

False

In [23]:
#Create a list of important columns for the recommendation_sytstem
columns = ['original_title','genre','director','actors']

In [27]:
new_df[columns].isnull().values.any()

False

In [28]:
#Create a function  to combine the values  of the important columns into a single string
def get_important_features(data):
    important_features = []
    for i  in range(0, data.shape[0]):
        important_features.append(data['original_title'][i]+' '+data['genre'][i]+' '+data['director'][i]+' '+data['actors'][i])
    
    return important_features

In [ ]:
# Creating column to the combined strings

new_df['important_features'] = get_important_features(new_df)

#Show the data
new_df.head(3)

In [ ]:
# Convert a text to a matrix of token counts
cm = CountVectorizer().fit_transform(df['improtant_features'])

In [ ]:
# Getting cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)
# Print the  cosine  similarity matrix
print(cs)


In [ ]:
# GET the shape of  the cosine simlarity  marix
cs.shape

In [ ]:
# Get the title of the movie  that the user likes
title = 'The Amazing  Spider-Man'

#Find the movie id
movie_id =df[df.original_title == title][Movie_id].values[0]

In [ ]:
#Cretae a list of enumerations for the similarity score [ (mocie_id, similarity_score), (....)]
scores = list(enumerate(cs[movie_id]))

In [ ]:
# Sort the list 
sorted_scores = sorted(scores, key = lamda x: x[1, reverse =True])
sorted_score = sorted_score[1:0]

In [ ]:
#Print the sorted scores
print(sorted_scores)

In [ ]:
#Create the loop  to print the first 7 most similar movies 
k=0
print('The 7 most recommeded movies to', title, 'are:\n')
for item in sorted_scores:
    movie_title = df[df.Movie_id == item[0]]['original_title']
    print(k+1, movie_title)
    k = k+1
    if k>4:
        break
    